## Baseline Dense Model



In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import time
from datetime import datetime

In [2]:
## script create_baseline_model.py
import tensorflow as tf
import numpy as np
from tensorflow import keras
import time
from datetime import datetime

def create_baseline_model():
    DIGIT_CLASSES = 10
    RESHAPED = 28 * 28 ## 784 pixesl per image
    #build the model
    model = tf.keras.models.Sequential()
    model.add(keras.layers.Dense(DIGIT_CLASSES,
                                 input_shape=(RESHAPED,),
                                 name='dense_layer',
                                 activation='softmax'))
    return model

def run_baseline_model(optimizer='SGD', verbose=0):
    """
    Create and run a dense Keras sequential model over the MNIST training data,
    using the specified numbers of hidden_layers, hidden_units_per_layer, and optimizer.
    Maybe add optional dropout TBD,
    Return values:
    history, training_time, test_accuracy, model
    """
    from keras.callbacks import EarlyStopping
    from load_mnist_dataset_flat import load_mnist_dataset_flat
    
    early_stopping = EarlyStopping(monitor='val_loss',verbose=1,patience=3,min_delta=0.0001)
    
    BATCH_SIZE = 128
    EPOCHS = 400
    VALIDATION_SPLIT = 0.2
    
    ## Load the training and test data
    X_train, X_test, Y_train, Y_test = load_mnist_dataset_flat()
    
    model = create_baseline_model()
    
    start_time = time.time()

    # compiling the model
    model.compile(optimizer=optimizer, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    #training the model
    history = model.fit(X_train, Y_train,
                        batch_size=BATCH_SIZE, epochs=EPOCHS,
                        verbose=verbose, validation_split=VALIDATION_SPLIT,
                        callbacks=[early_stopping]
                       )
    
    elapsed_time = time.time() - start_time
    
    test_loss, test_accuracy = model.evaluate(X_test, Y_test)
    print(f'training elapsed_time = {elapsed_time:10.1f}')
    print('Test accuracy:', test_accuracy)
    
    return  elapsed_time, test_accuracy

In [3]:
%run create_baseline_model
model = create_baseline_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_layer (Dense)         (None, 10)                7850      
                                                                 
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


Here we train the model 5 times to gather training runtime and test accuracy on the 10000 test samples. We want to know the test accuracy and some idea of how much it varies.

In [4]:
import pandas as pd
from datetime import datetime

## define functions to load the data, create the model, and run the model

%run create_baseline_model

results = []
for i in range(5):
    result = run_baseline_model()
    results.append(result)
    
df = pd.DataFrame(results)
df.columns = ['training_time_sec','test_accuracy']
display(df)
now = datetime.now()
result_name = now.strftime("data/BASELINE_DATA_%Y-%m-%d_%H_%M.csv")
df.to_csv(f'{result_name}')

60000 train samples
10000 test samples
Epoch 219: early stopping
313/313 [==============================] - 1s 3ms/step - loss: 0.2760 - accuracy: 0.9223
training elapsed_time =      233.9
Test accuracy: 0.9222999811172485
60000 train samples
10000 test samples
Epoch 218: early stopping
313/313 [==============================] - 1s 3ms/step - loss: 0.2767 - accuracy: 0.9221
training elapsed_time =      232.3
Test accuracy: 0.9221000075340271
60000 train samples
10000 test samples
Epoch 226: early stopping
313/313 [==============================] - 1s 3ms/step - loss: 0.2767 - accuracy: 0.9222
training elapsed_time =      240.8
Test accuracy: 0.9222000241279602
60000 train samples
10000 test samples
Epoch 215: early stopping
313/313 [==============================] - 1s 3ms/step - loss: 0.2766 - accuracy: 0.9221
training elapsed_time =      231.6
Test accuracy: 0.9221000075340271
60000 train samples
10000 test samples
Epoch 226: early stopping
313/313 [==============================] - 

,training_time_sec,test_accuracy
0,233.948682,0.9223
1,232.309703,0.9221
2,240.762712,0.9222
3,231.556649,0.9221
4,245.934716,0.9227


OSError: Cannot save file into a non-existent directory: 'data\data'

In [9]:
df.to_csv(f'{result_name}', index=False)
print (result_name)
display(df)

data/BASELINE_DATA_2022-09-30_18_38.csv


,training_time_sec,test_accuracy
0,233.948682,0.9223
1,232.309703,0.9221
2,240.762712,0.9222
3,231.556649,0.9221
4,245.934716,0.9227


In [7]:
df.describe()

,training_time_sec,test_accuracy
count,5.000000,5.000000
mean,236.902493,0.922280
std,6.222522,0.000249
min,231.556649,0.922100
25%,232.309703,0.922100
50%,233.948682,0.922200
75%,240.762712,0.922300
max,245.934716,0.922700
